In [13]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import pandas as pd
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# display 파라미터는 default parameter이다.
def search_books(query, display=50): 
    # query string 문자열을 dict 선언
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }

    url = 'https://openapi.naver.com/v1/search/book.json' #?query=파이썬&display=100&sort=sim

    # requests get(url, params, headers) 요청 
    res = requests.get(url, params=payload, headers=headers)
    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']

    return items_data  #[{},{}]

# def save_json(items_data):
#     with open('../data/books.json','w',encoding='utf-8') as file:
#         json.dump(items_data, file)

def search_shops(query, display=50): 
    # query string 문자열을 dict 선언
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }

    url = 'https://openapi.naver.com/v1/search/shop.json' #?query=파이썬&display=100&sort=sim

    res = requests.get(url, params=payload, headers=headers)
    # json() 함수로 응답 결과 가져오기
    items_data = res.json()['items']
    return items_data

def save_json(items_data):
    with open('../data/books.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)

def save_json_shop(items_data):
    with open('../data/shops.json','w',encoding='utf-8') as file:
        json.dump(items_data, file)


if __name__ == '__main__':
    save_json(search_books('파이썬'))        
    save_json_shop(search_shops('가디건'))

### 리팩토링 된 코드

In [14]:
import requests
import os
from dotenv import load_dotenv
import json

# .env 파일에서 환경 변수 로드
load_dotenv()

client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}


def search_naver_api(endpoint, query, display=50):
    """네이버 API 검색 함수"""
    payload = {
        'query': query,
        'display': display,
        'sort': 'sim'
    }
    url = f'https://openapi.naver.com/v1/search/{endpoint}.json'
    res = requests.get(url, params=payload, headers=headers)
    res.raise_for_status()  # 에러 발생 시 예외 처리
    return res.json().get('items', [])


def save_json(data, filepath):
    """JSON 파일 저장 함수"""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)


if __name__ == '__main__':
    books = search_naver_api('book', '파이썬')
    save_json(books, '../data/books.json')

    shops = search_naver_api('shop', '가디건')
    save_json(shops, '../data/shops.json')


In [15]:
import pandas as pd

books_df = pd.read_json('../data/books.json')
print(type(books_df))
books_df.head(2)

<class 'pandas.core.frame.DataFrame'>


,title,link,image,author,discount,publisher,pubdate,isbn,description
0,처음이야? 파이썬 기초 (동영상 강의로 배우는 292개 코드 따라하기(핵심노트+오픈...,https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_550256...,윤영빈^오환^이용희,18000,영진닷컴,20250620,9788931478006,파이썬 완전 초보에게 추천합니다.\n파이썬을 처음 배우는 당신을 위한 가장 쉬운 기...
1,모두의 인공지능 with 파이썬 (누구나 쉽게 시작하는 AI 기초 프로그래밍),https://search.shopping.naver.com/book/catalog...,https://shopping-phinf.pstatic.net/main_558136...,이영호,27000,길벗,20250725,9791140714995,"가장 쉬운 인공지능 입문서, \n생성형 AI를 반영해 한층 더 탄탄하게 돌아왔다!\..."


In [16]:

books_df.loc[books_df['discount'] >= 20000,['title','author','discount','publisher','pubdate']]\
    .sort_values(by='discount', ascending=False).reset_index(drop=True)

,title,author,discount,publisher,pubdate
0,으뜸 파이썬 (개정판),박동규^강영민,32300,생능출판,20240614
1,으뜸 파이썬 (프로그래밍을 사랑하는 두 교수가 작정하고 쓴),박동규^강영민,31680,생능출판,20200217
2,파이썬,Y. Daniel Liang,31500,에피스테메,20180302
3,파이썬 마스터 (실생활 융합 예제로 배우는),김종훈^김동건,31040,한빛아카데미,20250117
4,파이썬,홍의경,30400,생능출판,20220309
5,독학 파이썬,야마다 요시히로,28800,정보문화사,20241220
6,새내기 파이썬,천인국,28500,생능출판,20220630
7,모두의 인공지능 with 파이썬 (누구나 쉽게 시작하는 AI 기초 프로그래밍),이영호,27000,길벗,20250725
8,파이썬의 정석,조용주^임좌상,26820,길벗캠퍼스,20230110
9,내공의 파이썬 (한 권으로 끝내는 초보자용 코딩 입문서),한선관,24300,자유아카데미,20250315


In [17]:
# 함수로 정의
def filter_and_sort_books(df, min_discount=20000):
    """
    할인 금액이 min_discount 이상인 도서 필터링 후 정렬
    
    Parameters:
        df (DataFrame): 도서 데이터프레임
        min_discount (int): 최소 할인 금액 기준 (기본값 20000)

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['discount'] >= min_discount, ['title', 'author', 'discount', 'publisher', 'pubdate']]
          .sort_values(by='discount', ascending=False)
          .reset_index(drop=True)
    )

filter_and_sort_books(books_df,30000)

,title,author,discount,publisher,pubdate
0,으뜸 파이썬 (개정판),박동규^강영민,32300,생능출판,20240614
1,으뜸 파이썬 (프로그래밍을 사랑하는 두 교수가 작정하고 쓴),박동규^강영민,31680,생능출판,20200217
2,파이썬,Y. Daniel Liang,31500,에피스테메,20180302
3,파이썬 마스터 (실생활 융합 예제로 배우는),김종훈^김동건,31040,한빛아카데미,20250117
4,파이썬,홍의경,30400,생능출판,20220309


In [18]:
books_df.columns

Index(['title', 'link', 'image', 'author', 'discount', 'publisher', 'pubdate',
       'isbn', 'description'],
      dtype='object')

In [19]:
# image , description 컬럼은 제외한 모든 컬럼 출력하기
books_df.loc[books_df['publisher'].str.contains('인피니티북스'),\
    books_df.columns.drop(['image','description'])].reset_index(drop=True)

,title,link,author,discount,publisher,pubdate,isbn
0,파이썬,https://search.shopping.naver.com/book/catalog...,천인국,10800,인피니티북스,20170830,9791185578330
1,파이썬 플러스,https://search.shopping.naver.com/book/catalog...,최희식,22500,인피니티북스,20240731,9791192373362


In [20]:
books_df['publisher'].unique()

array(['영진닷컴', '길벗', '한빛미디어', '이지스퍼블리싱', '생능출판', '인피니티북스', '렉스미디어닷넷',
       '북두', '한빛아카데미', '에듀웨이', '기한재', '복두출판사', '다본', '에피스테메', '정보문화사',
       '자유아카데미', '한국방송통신대학교출판문화원', '디지털북스', '퍼플', '북랩', '그린', '에이콘출판',
       '시그마프레스', '지오북스', '이모션미디어', '로드북', '클라우드북스', '성안당', '연두에디션',
       '길벗캠퍼스'], dtype=object)

In [21]:
def filter_books_by_publisher(df, publisher_name):
    """
    특정 출판사가 포함된 도서만 필터링 (image, description 컬럼 제외)

    Parameters:
        df (DataFrame): 도서 데이터프레임
        publisher_name (str): 포함할 출판사 이름

    Returns:
        DataFrame: 필터링된 결과
    """
    return (
        df.loc[df['publisher'].str.contains(publisher_name), df.columns.drop(['image', 'description'])]
          .reset_index(drop=True)
    )

filter_books_by_publisher(books_df,'한빛미디어')

,title,link,author,discount,publisher,pubdate,isbn
0,혼자 공부하는 파이썬 (1:1 과외하듯 배우는 프로그래밍 자습서),https://search.shopping.naver.com/book/catalog...,윤인성,19800,한빛미디어,20220601,9791162245651
1,밑바닥부터 시작하는 딥러닝 1(리마스터판) (파이썬으로 익히는 딥러닝 이론과 구현),https://search.shopping.naver.com/book/catalog...,사이토 고키,23400,한빛미디어,20250124,9791169213387
2,파이썬 3 (프로그래밍을 배우기에 가장 재미있는 언어),https://search.shopping.naver.com/book/catalog...,박상현,0,한빛미디어,20160210,9788968482359
3,혼자 공부하는 데이터 분석 with 파이썬 (1:1 과외하듯 배우는 데이터 분석 자습서),https://search.shopping.naver.com/book/catalog...,박해선,23400,한빛미디어,20230102,9791169210287
4,파이썬 정복 (파이썬 개발에 필요한 기본 지식을 모두 담은 파이썬 입문서),https://search.shopping.naver.com/book/catalog...,김상형,19800,한빛미디어,20180402,9791162240540


In [22]:
import pandas as pd

shops_df = pd.read_json('../data/shops.json')
print(type(shops_df))
shops_df.head(2)

<class 'pandas.core.frame.DataFrame'>


,title,link,image,lprice,hprice,mallName,productId,productType,brand,maker,category1,category2,category3,category4
0,브이넥 얇은 비치 볼레로<b>가디건</b> 휴양지 쿨 망사 긴팔 시스루 봄 크롭 니...,https://smartstore.naver.com/main/products/744...,https://shopping-phinf.pstatic.net/main_114505...,17900,,센데로,11450534248,2,,,패션의류,여성의류,카디건,
1,썸머 <b>가디건</b> 여성 여름볼레로 <b>가디건</b> 여름 크롭 긴팔 얇은 시스루,https://smartstore.naver.com/main/products/854...,https://shopping-phinf.pstatic.net/main_860940...,13900,,바띠소띠,86094096996,2,바띠소띠,,패션의류,여성의류,카디건,


In [23]:
shops_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        50 non-null     object
 1   link         50 non-null     object
 2   image        50 non-null     object
 3   lprice       50 non-null     int64 
 4   hprice       50 non-null     object
 5   mallName     50 non-null     object
 6   productId    50 non-null     int64 
 7   productType  50 non-null     int64 
 8   brand        50 non-null     object
 9   maker        50 non-null     object
 10  category1    50 non-null     object
 11  category2    50 non-null     object
 12  category3    50 non-null     object
 13  category4    50 non-null     object
dtypes: int64(3), object(11)
memory usage: 5.6+ KB


In [24]:
shops_df.loc[shops_df['lprice'] <= 50000,['brand','lprice','mallName','link']]\
    .sort_values(by='lprice').reset_index(drop=True)

,brand,lprice,mallName,link
0,,5790,네이버,https://search.shopping.naver.com/catalog/4539...
1,,8400,눈키스타,https://smartstore.naver.com/main/products/119...
2,르샵,9900,네이버,https://search.shopping.naver.com/catalog/5434...
3,지오다노,12580,네이버,https://search.shopping.naver.com/catalog/5513...
4,,13500,오다오,https://smartstore.naver.com/main/products/253...
5,바띠소띠,13900,바띠소띠,https://smartstore.naver.com/main/products/854...
6,,14900,WITH SEN,https://smartstore.naver.com/main/products/863...
7,,14900,슈퍼캣파랑,https://smartstore.naver.com/main/products/100...
8,피카부,15300,베베드키즈,https://smartstore.naver.com/main/products/478...
9,스파오,15300,네이버,https://search.shopping.naver.com/catalog/4835...


In [25]:
def filter_and_sort_shops(df, max_price=50000):
    """
    최대 가격 이하 상품 필터링 후 가격 기준 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        max_price (int): 최대 가격 기준 (기본값 50000)

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['lprice'] <= max_price, ['brand', 'lprice', 'mallName', 'link']]
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_and_sort_shops(shops_df,20000)

,brand,lprice,mallName,link
0,,5790,네이버,https://search.shopping.naver.com/catalog/4539...
1,,8400,눈키스타,https://smartstore.naver.com/main/products/119...
2,르샵,9900,네이버,https://search.shopping.naver.com/catalog/5434...
3,지오다노,12580,네이버,https://search.shopping.naver.com/catalog/5513...
4,,13500,오다오,https://smartstore.naver.com/main/products/253...
5,바띠소띠,13900,바띠소띠,https://smartstore.naver.com/main/products/854...
6,,14900,슈퍼캣파랑,https://smartstore.naver.com/main/products/100...
7,,14900,WITH SEN,https://smartstore.naver.com/main/products/863...
8,피카부,15300,베베드키즈,https://smartstore.naver.com/main/products/478...
9,스파오,15300,네이버,https://search.shopping.naver.com/catalog/4835...


In [26]:
shops_df.columns

Index(['title', 'link', 'image', 'lprice', 'hprice', 'mallName', 'productId',
       'productType', 'brand', 'maker', 'category1', 'category2', 'category3',
       'category4'],
      dtype='object')

In [27]:
shops_df.loc[shops_df['mallName'] == '네이버','lprice':'brand']\
    .sort_values(by='lprice').reset_index(drop=True)

,lprice,hprice,mallName,productId,productType,brand
0,5790,,네이버,45390721179,1,
1,9900,,네이버,54347429849,1,르샵
2,12580,,네이버,55136646682,1,지오다노
3,15300,,네이버,48350506813,1,스파오
4,17930,,네이버,54348404387,1,지오다노
5,18000,,네이버,54708238294,1,
6,18300,,네이버,46159571697,1,
7,21080,,네이버,55538923583,1,지오다노
8,21520,,네이버,53629413300,1,스파오
9,24770,,네이버,54815070305,1,제로스트릿


In [28]:
shops_df['mallName'].unique()

array(['센데로', '바띠소띠', '나우인뉴욕', '코이블리', '눈키스타', '네이버', 'ARUMY', '슈퍼캣파랑',
       '멜빈트 MELVINT', 'RENCY', '로우라이트', '벨라엘라', '고고하니', '디스룩', '베베드키즈',
       '아크민', 'WITH SEN', '무드멜리', '빈콜렉터', '오다오', '올내츄럴', '미스모네', '234',
       '리틀데이즈', '러브썸원', '파인 스트리트', 'ttoyuni'], dtype=object)

In [29]:
shops_df['brand'].unique()

array(['', '바띠소띠', '폴로랄프로렌', '타미진스', '지오다노', '스파오', '제너럴아이디어', '제로스트릿',
       '피카부', '빈콜렉터', '르샵', '라코스테', '후아유', '쉬즈미스', '미스모네', '메종키츠네',
       '씨씨콜렉트', 'ZOOC', '온앤온', '러브썸원'], dtype=object)

In [30]:
def filter_shops_by_mall(df, mall_name='네이버'):
    """
    특정 쇼핑몰 상품만 필터링 후 가격 기준 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        mall_name (str): 쇼핑몰 이름 (기본값 '네이버')

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    return (
        df.loc[df['mallName'] == mall_name, 'lprice':'brand']
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_shops_by_mall(shops_df,'나우인뉴욕')

,lprice,hprice,mallName,productId,productType,brand
0,63800,,나우인뉴욕,84216650390,2,폴로랄프로렌


#### mallName 또는 brand 로 검색하기

In [32]:
def filter_shops(df, keyword, search_type='mall'):
    """
    mallName 또는 brand 기준으로 필터링 후 가격 오름차순 정렬

    Parameters:
        df (DataFrame): 쇼핑 데이터프레임
        keyword (str): 검색할 값 (예: '네이버', '폴로랄프로렌')
        search_type (str): 'mall' or 'brand' (기본값 'mall')

    Returns:
        DataFrame: 필터링 및 정렬된 결과
    """
    
    if search_type == 'mall':
        condition = df['mallName'] == keyword
    elif search_type == 'brand':
        condition = df['brand'] == keyword
    else:
        raise ValueError("search_type은 'mall' 또는 'brand'만 가능합니다.")

    return (
        df.loc[condition, 'lprice':'brand']
          .sort_values(by='lprice')
          .reset_index(drop=True)
    )

filter_shops(shops_df,'폴로랄프로렌','brand')

,lprice,hprice,mallName,productId,productType,brand
0,62900,,파인 스트리트,84238485864,2,폴로랄프로렌
1,63800,,나우인뉴욕,84216650390,2,폴로랄프로렌
2,69000,,리틀데이즈,87004045841,2,폴로랄프로렌
